In [1]:
from functools import partial

import jax
import jax.numpy as jnp
import jax.random as jr
import numpy as np
import numpyro
import ot
import scipy
from jax import Array
from numpyro import diagnostics, distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro.infer.util import initialize_model

from mcmc import run_lmc


jnp.set_printoptions(precision=3, suppress=True)
jax.config.update("jax_enable_x64", True)


def get_model_and_data(data, name):
    dset = data[name][0, 0]
    x = dset["x"]
    labels = dset["t"]
    n, data_dim = x.shape
    n_train = min(int(n / 2), 500)
    x_train = x[:n_train]
    labels_train = labels[:n_train]
    x_test = x[n_train:]
    labels_test = labels[n_train:]

    def model():
        alpha = numpyro.sample("alpha", dist.Exponential(0.01))
        W = numpyro.sample("W", dist.Normal(jnp.zeros(data_dim), 1.0 / alpha))
        logits = jnp.sum(W * x_train, axis=-1)
        return numpyro.sample("obs", dist.Bernoulli(logits=logits), obs=labels_train)

    return model, (x_train, labels_train, x_test, labels_test)


data = scipy.io.loadmat("mcmc_data/benchmarks.mat")

data_name = "diabetis"
model_logreg, data_split = get_model_and_data(data, data_name)

logreg_info = initialize_model(jr.PRNGKey(0), model_logreg)


def dict_to_array(dct: dict):
    alpha = dct["alpha"]

    return jnp.concatenate([jnp.expand_dims(alpha, alpha.ndim), dct["W"]], axis=-1)


vec_dict_to_array = jax.jit(jax.vmap(dict_to_array, in_axes=0, out_axes=0))


@jax.jit
def array_to_dict(arr: jnp.ndarray):
    return {"alpha": arr[0], "W": arr[1:]}


@jax.jit
def potential_fn(arr: jnp.ndarray):
    dct = array_to_dict(arr)
    return logreg_info.potential_fn(dct)


arr0 = dict_to_array(logreg_info.param_info.z)

print(jax.devices("cuda"))


def compute_w2(x1, x2, num_iters):
    source_samples = np.array(x1)
    target_samples = np.array(x2)
    source_weights = np.ones(source_samples.shape[0]) / source_samples.shape[0]
    target_weights = np.ones(target_samples.shape[0]) / target_samples.shape[0]
    mm = ot.dist(source_samples, target_samples)
    return ot.emd2(source_weights, target_weights, mm, numItermax=num_iters)


@partial(jax.jit, static_argnames=("max_len",))
def energy_distance(x: Array, y: Array, max_len: int = 2**15):
    assert y.ndim == x.ndim
    assert x.shape[1:] == y.shape[1:]
    if x.shape[0] > max_len:
        x = x[:max_len]
    if y.shape[0] > max_len:
        y = y[:max_len]

    @partial(jax.vmap, in_axes=(None, 0))
    def _dist_single(_x, _y_single):
        assert _x.ndim == _y_single.ndim + 1, f"{_x.ndim} != {_y_single.ndim + 1}"
        diff = _x - _y_single
        if x.ndim > 1:
            # take the norm over all axes except the first one
            diff = jnp.sqrt(jnp.sum(diff**2, axis=tuple(range(1, diff.ndim))))
        return jnp.mean(jnp.abs(diff))

    def dist(_x, _y):
        assert _x.ndim == _y.ndim
        return jnp.mean(_dist_single(_x, _y))

    return 2 * dist(x, y) - dist(x, x) - dist(y, y)


def eval_logreg(samples, evals_per_sample, ground_truth=None, num_iters_w2=0):
    if isinstance(samples, dict):
        samples = vec_dict_to_array(samples)
    ess = diagnostics.effective_sample_size(samples)
    print(f"Effective sample size: {ess}")
    # print(
    #     f"Gradient evals per effective sample:"
    #     f" {evals_per_sample * samples.shape[0]/ess}"
    # )
    if ground_truth is None:
        return
    sample_dim = samples.shape[-1]
    reshaped = jnp.reshape(samples, (-1, sample_dim))
    energy_gt = energy_distance(reshaped, ground_truth)
    half_len = reshaped.shape[0] // 2
    energy_self = energy_distance(reshaped[:half_len], reshaped[half_len:])
    result_str = (
        f"Energy dist vs ground truth: {energy_gt:.4}, vs self: {energy_self:.4}"
    )
    if num_iters_w2 > 0:
        w2 = compute_w2(reshaped, ground_truth, num_iters_w2)
        result_str += f", Wasserstein-2: {w2:.4}"
    print(result_str)

[cuda(id=0)]


In [2]:
gt_nuts = MCMC(NUTS(model_logreg, step_size=2**-4), num_warmup=2**10, num_samples=2**15)
gt_nuts.run(jr.PRNGKey(0))
gt_logreg = vec_dict_to_array(gt_nuts.get_samples())
np.save(f"mcmc_data/{data_name}_ground_truth.npy", gt_logreg)
gt_logreg = np.load(f"mcmc_data/{data_name}_ground_truth.npy")
size_gt_half = int(gt_logreg.shape[0] // 2)
energy_bias = energy_distance(gt_logreg[:size_gt_half], gt_logreg[size_gt_half:])
print(f"Energy bias: {energy_bias:.4}")

sample:  17%|█▋        | 5783/33792 [27:53<2:15:07,  3.45it/s, 1023 steps of size 1.92e-15. acc. prob=0.92]


KeyboardInterrupt: 

In [18]:
num_chains = 2**7
num_samples_per_chain = 2**8
out_logreg_lmc, steps_logreg_lmc = run_lmc(
    jr.PRNGKey(0),
    potential_fn,
    arr0,
    num_chains,
    num_samples_per_chain,
    chain_sep=0.25,
    tol=0.1,
    warmup_mult=128.0,
    warmup_tol_mult=4.0,
)
print(out_logreg_lmc.shape)

LMC: Steps warmup: 2300.2734375, steps mcmc: 24664.6640625, gradient evaluations per output: 210.66357421875
(128, 256, 9)


In [23]:
eval_logreg(out_logreg_lmc, steps_logreg_lmc, gt_logreg)

Effective sample size: [427.2759 323.3381 323.3381 323.3381 323.3381 323.3381 323.3381 323.3382
 323.3381]:.4
Gradient evals per effective sample: [63.109  83.3955 83.3955 83.3955 83.3955 83.3955 83.3955 83.3955 83.3955]
Energy distance vs ground truth: 1.216e+08, energy distance vs self: 0.007298


In [8]:
nuts = MCMC(
    NUTS(model_logreg),
    num_warmup=2**3,
    num_samples=num_samples_per_chain,
    num_chains=num_chains,
    chain_method="vectorized",
)
nuts.run(jr.PRNGKey(0), extra_fields=("num_steps",))
out_logreg_nuts = nuts.get_samples(group_by_chain=True)
num_steps_nuts = nuts.get_extra_fields()["num_steps"]
geps_nuts = sum(num_steps_nuts) / len(num_steps_nuts)
print(geps_nuts)

sample: 100%|██████████| 264/264 [00:59<00:00,  4.44it/s]


10.700408935546875


In [9]:
eval_logreg(out_logreg_nuts, geps_nuts, gt_logreg)

Effective sample size: [    nan 65.1355     nan]
Gradient evals per effective sample: [    nan 21.0277     nan]
W2 distance: 0.03975801461724085


/home/andy/PycharmProjects/diffrax_STLA/venv/lib/python3.11/site-packages/ot/lp/__init__.py:571: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  check_result(result_code)


In [2]:
import os


def run_logreg_dataset(name):
    model_logreg, data_split = get_model_and_data(data, name)
    logreg_info = initialize_model(jr.PRNGKey(0), model_logreg)
    arr0 = dict_to_array(logreg_info.param_info.z)

    @jax.jit
    def _potential_fn(arr: jnp.ndarray):
        dct = array_to_dict(arr)
        return logreg_info.potential_fn(dct)

    # Compute ground truth
    ground_truth_filename = f"mcmc_data/{name}_ground_truth.npy"
    if os.path.isfile(ground_truth_filename):
        ground_truth = np.load(ground_truth_filename)
    else:
        gt_nuts = MCMC(
            NUTS(model_logreg, step_size=0.125), num_warmup=2**13, num_samples=2**15
        )
        gt_nuts.run(jr.PRNGKey(0))
        ground_truth = vec_dict_to_array(gt_nuts.get_samples())
        np.save(f"mcmc_data/{name}_ground_truth.npy", ground_truth)
    size_gt_half = int(ground_truth.shape[0] // 2)
    gt_energy_bias = energy_distance(
        ground_truth[:size_gt_half], ground_truth[size_gt_half:]
    )
    print(f"Ground truth energy bias: {gt_energy_bias:.4}")
    num_chains = 2**7
    num_samples_per_chain = 2**8

    out_logreg_lmc, steps_logreg_lmc = run_lmc(
        jr.PRNGKey(0),
        _potential_fn,
        arr0,
        num_chains,
        num_samples_per_chain,
        chain_sep=0.25,
        tol=0.1,
        warmup_mult=128.0,
        warmup_tol_mult=8.0,
    )
    eval_logreg(out_logreg_lmc, steps_logreg_lmc, ground_truth)

    nuts = MCMC(
        NUTS(model_logreg),
        num_warmup=2**8,
        num_samples=num_samples_per_chain,
        num_chains=num_chains,
        chain_method="vectorized",
    )
    nuts.run(jr.PRNGKey(0), extra_fields=("num_steps",))
    out_logreg_nuts = nuts.get_samples(group_by_chain=True)
    num_steps_nuts = nuts.get_extra_fields()["num_steps"]
    geps_nuts = sum(num_steps_nuts) / len(num_steps_nuts)
    print(f"NUTS: Gradient evals per output: {geps_nuts}")
    eval_logreg(out_logreg_nuts, geps_nuts, ground_truth)

In [3]:
names = [
    "banana",
    "breast_cancer",
    "diabetis",
    "flare_solar",
    "german",
    "heart",
    "image",
    "ringnorm",
    "splice",
    "thyroid",
    "titanic",
    "twonorm",
    "waveform",
]

In [4]:
import warnings


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for name in names:
        print(f"==================== {name} ====================")
        run_logreg_dataset(name)

==================== banana ====================
LMC
LMC: Steps warmup: 600.5859375, steps mcmc: 9082.40625, gradient evaluations per output: 75.64837646484375
Effective sample size: [  528.9647  3286.149  13097.2888]
Gradient evals per effective sample: [18.3056  2.9466  0.7393]
W2 distance: 0.023220977744452573
NUTS


sample: 100%|██████████| 320/320 [00:57<00:00,  5.56it/s]


Gradient evals per effective sample (NUTS): 13.71722412109375
Effective sample size: [ 6194.9014 27028.3926 24080.6835]
Gradient evals per effective sample: [0.2834 0.065  0.0729]
W2 distance: 0.00028803654533254326
LMC
==================== breast_cancer ====================
LMC
LMC: Steps warmup: 25.6640625, steps mcmc: 910.875, gradient evaluations per output: 7.31671142578125
Effective sample size: [4788.6521  323.302   323.3166  323.2347  323.3078  323.2946  323.2509
  323.4151  319.7926  323.3168]
Gradient evals per effective sample: [0.1956 2.8968 2.8967 2.8974 2.8967 2.8969 2.8973 2.8958 2.9286 2.8967]
W2 distance: 540404996204079.0
NUTS


sample: 100%|██████████| 320/320 [00:32<00:00,  9.76it/s]


Gradient evals per effective sample (NUTS): 8.668121337890625
Effective sample size: [    nan     nan 65.2196     nan     nan     nan     nan     nan     nan
     nan]
Gradient evals per effective sample: [   nan    nan 17.012    nan    nan    nan    nan    nan    nan    nan]
W2 distance: 314604860600664.06
LMC
==================== diabetis ====================


...

Gradient evals per effective sample (NUTS): 3.792510986328125
Effective sample size: [     nan      nan      nan 129.4707      nan      nan      nan      nan
      nan 129.4675      nan 129.4731      nan      nan 129.4689      nan
      nan 129.4727      nan      nan 129.4715]
Gradient evals per effective sample: [   nan    nan    nan 3.7494    nan    nan    nan    nan    nan 3.7495
    nan 3.7494    nan    nan 3.7495    nan    nan 3.7494    nan    nan
 3.7494]
W2 distance: 1.4843556079383466e+24
LMC
==================== waveform ====================


sample: 100%|██████████| 16640/16640 [1:23:37<00:00,  3.32it/s, 1023 steps of size 2.15e-15. acc. prob=0.19]


LMC
LMC: Steps warmup: 30.0, steps mcmc: 781.7578125, gradient evaluations per output: 6.34185791015625
Effective sample size: [1392.787   323.3003  323.3005  323.2975  323.3502  323.3044  323.303
  323.2576  323.2774  323.302   323.3014  323.2743  323.3023  323.3026
  323.302   323.3028  323.3014  323.3064  323.3054  323.2991  323.2992
  323.3093]
Gradient evals per effective sample: [0.5828 2.5108 2.5108 2.5109 2.5105 2.5108 2.5108 2.5112 2.511  2.5108
 2.5108 2.511  2.5108 2.5108 2.5108 2.5108 2.5108 2.5108 2.5108 2.5109
 2.5109 2.5108]
W2 distance: 9.646766406379034e+21
NUTS


sample: 100%|██████████| 320/320 [00:47<00:00,  6.78it/s]


Gradient evals per effective sample (NUTS): 3.255615234375
Effective sample size: [    nan     nan     nan     nan     nan     nan     nan 64.1405     nan
 73.1303     nan     nan     nan 73.0969     nan     nan 73.1399     nan
 72.8051     nan     nan     nan]
Gradient evals per effective sample: [   nan    nan    nan    nan    nan    nan    nan 6.497     nan 5.6983
    nan    nan    nan 5.7009    nan    nan 5.6976    nan 5.7238    nan
    nan    nan]
W2 distance: 1.028961149034422e+22
LMC
